In [ ]:
%matplotlib inline

In [ ]:
import sys, os
import csv
import matplotlib
#matplotlib.use('Agg')
#import collections

import datetime
import toml
import netCDF4
import numpy as np
import matplotlib.pyplot as plt

import spectra_mole
from spectra_mole import h

In [ ]:
files = {'spec': '../../colrawi/cloudradar/spectra/D20150617_T0000_0000_Lin_zspc2nc_v1_02_standard.nc4',
         'mmclx': '../../colrawi/cloudradar/mmclx/20150617_0000.mmclx',
         'rwp': '../../colrawi/windprofiler/nc/wp_20150617_043156.nc.iop',
         'cloudnet': '../../colrawi/cloudnet/20150617_lindenberg_categorize.nc'}

toml_file = '../filenames.toml'
with open(toml_file) as f:
    all_files = toml.loads(f.read())


all_files['20150617a']
    

In [ ]:
def load_csv():
    filename = '../cases.csv'
    #filename = '../cases_LDR.csv'
    filename = '../cases_calibration.csv'
    data = []
    with open(filename, 'r') as f:
        reader = csv.reader(f, delimiter=';')
        header = next(reader)
        print(header)
        for row in reader:
            if not row[0][0] == '#':
                d = {k:v for k, v in zip(header, row)}
                d['dt'] = datetime.datetime.strptime(d['datetime'], '%Y-%m-%d-%H:%M:%S')
                data.append(d)
    return data

csv_file = load_csv()

for case in csv_file:
    dt = case['dt']
    rg = float(case['range'])    
    files = all_files[case['file_specifier']]
    cr = spectra_mole.mira(files)
    rwp = spectra_mole.rwp(files)
    advect = spectra_mole.advection.cloudnet_advect(files['cloudnet'])
    
    corr = spectra_mole.correct_pixel(cr, rwp, advect, h.dt_to_ts(dt), rg, visualize=True)
    print('flag ', corr['flag'], corr['bflag'])
    print("cal corr", corr["cal_corr"], corr["cal_const"])
    for k in corr.keys():
        print(k)
        if type(corr[k]) is dict:
            for kn in corr[k].keys():
                print('  ', kn)
    if 'spec_broad' in corr.keys():
        print('cr broad ', corr['spec_broad']['moments'])
